In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, BaggingRegressor
from prophet import Prophet

train = pd.read_csv('./train.csv')
# 결과를 저장하기 위한 dataframe 생성
results_df = pd.DataFrame(columns=['종목코드', 'final_return'])

# train 데이터에 존재하는 독립적인 종목코드 추출
unique_codes = train['종목코드'].unique()

# 과거 데이터를 기반으로 데이터프레임을 구성하는 함수
def create_features(df, n_days):
    df_new = df.copy()
    for i in range(1, n_days+1):
        df_new[f'종가_lag_{i}'] = df_new['종가'].shift(i)
    df_new.dropna(inplace=True)
    return df_new

# 각 종목코드에 대해서 모델 학습 및 추론 반복
for code in tqdm(unique_codes):
    
    # 학습 데이터 생성
    train_data = train[train['종목코드'] == code][['일자', '종가']]
    train_data['일자'] = pd.to_datetime(train_data['일자'], format='%Y%m%d')
    train_data.set_index('일자', inplace=True)
    
    # 과거 5일의 데이터를 특성으로 사용
    train_data = create_features(train_data, 5)
    X = train_data.drop('종가', axis=1).values
    y = train_data['종가'].values
    
    # 데이터 분할
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)
    
    # GB 모델 선언 및 학습
    base_model = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1)
    model = BaggingRegressor(base_estimator=base_model, n_estimators=10, random_state=0)
    model.fit(X_train, y_train)
    
    # Prophet 모델 선언 및 학습
    df_prophet = pd.DataFrame(y_train).reset_index()
    df_prophet.columns = ['ds', 'y']
    model_prophet = Prophet()
    model_prophet.fit(df_prophet)
    
    # 향후 15개의 거래일에 대해 GB 및 Prophet 추론
    predictions = []
    predictions_prophet = []
    X_last = X_test[-1, :]
    future = model_prophet.make_future_dataframe(periods=15)
    forecast = model_prophet.predict(future)['yhat'][-15:]
    for _ in range(15):
        pred = model.predict(X_last.reshape(1, -1))
        predictions.append(pred[0])
        predictions_prophet.append(forecast[_])
        X_last = np.roll(X_last, -1)
        X_last[-1] = pred
    
    # GB와 Prophet의 예측 결과를 평균내어 최종 수익률 계산
    predictions_avg = [(predictions[i] + predictions_prophet[i]) / 2 for i in range(15)]
    final_return = (predictions_avg[-1] - predictions_avg[0]) / predictions_avg[0]
    
    # 결과 저장
    results_df = results_df.append({'종목코드': code, 'final_return': final_return}, ignore_index=True)

# 각 순위를 중복없이 생성
results_df['순위'] = results_df['final_return'].rank(method='first', ascending=False).astype('int')
results_df


  0%|          | 0/2000 [00:00<?, ?it/s]


ParserError: day is out of range for month: 0 present at position 0